In [121]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [122]:
movies = pd.read_csv('/kaggle/input/movie-dataset/TMBD Movie Dataset.csv')

In [123]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1287 entries, 0 to 1286
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            1287 non-null   int64  
 1   id                    1287 non-null   int64  
 2   imdb_id               1287 non-null   object 
 3   popularity            1287 non-null   float64
 4   budget                1287 non-null   float64
 5   revenue               1287 non-null   float64
 6   original_title        1287 non-null   object 
 7   cast                  1287 non-null   object 
 8   homepage              1287 non-null   object 
 9   director              1287 non-null   object 
 10  tagline               1287 non-null   object 
 11  keywords              1287 non-null   object 
 12  overview              1287 non-null   object 
 13  runtime               1287 non-null   int64  
 14  genres                1287 non-null   object 
 15  production_companies 

In [124]:
movies.head()

,Unnamed: 0,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,...,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj,profit,popularity_level
0,0,135397,tt0369610,32.985763,150000000.0,1.513529e+09,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,...,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,2015-06-09,5562,6.5,2015,1.379999e+08,1.392446e+09,1.363529e+09,High
1,1,76341,tt1392190,28.419936,150000000.0,3.784364e+08,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,...,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,2015-05-13,6185,7.1,2015,1.379999e+08,3.481613e+08,2.284364e+08,High
2,2,262500,tt2908446,13.112507,110000000.0,2.952382e+08,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,...,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,2015-03-18,2480,6.3,2015,1.012000e+08,2.716190e+08,1.852382e+08,High
3,3,140607,tt2488496,11.173104,200000000.0,2.068178e+09,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,...,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,2015-12-15,5292,7.5,2015,1.839999e+08,1.902723e+09,1.868178e+09,High
4,4,168259,tt2820852,9.335014,190000000.0,1.506249e+09,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,...,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,2015-04-01,2947,7.3,2015,1.747999e+08,1.385749e+09,1.316249e+09,High


In [125]:
movies.shape

(1287, 24)

In [126]:
#id
#popularity
#original_title
#cast
#director
#genres
#release_date
#release_year
#popularity

In [127]:
selected_columns = ['id', 'original_title', 'overview', 'genres', 'keywords', 'cast', 'director']

In [128]:
movies = movies[selected_columns]

In [129]:
movie.shape

(1287, 8)

In [130]:
# Select relevant features.
selected_columns = ['id', 'original_title', 'overview', 'genres', 'keywords', 'cast', 'director']

In [131]:
movie.shape

(1287, 8)

In [132]:
# Check if columns exist before selecting to avoid KeyError
existing_cols = [col for col in selected_columns if col in movies.columns]
movies = movies[existing_cols]

In [133]:
movie.head()

,id,popularity,original_title,cast,director,genres,release_date,popularity_level
0,135397,32.985763,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,Action|Adventure|Science Fiction|Thriller,2015-06-09,High
1,76341,28.419936,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,Action|Adventure|Science Fiction|Thriller,2015-05-13,High
2,262500,13.112507,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,Adventure|Science Fiction|Thriller,2015-03-18,High
3,140607,11.173104,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Action|Adventure|Science Fiction|Fantasy,2015-12-15,High
4,168259,9.335014,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Action|Crime|Thriller,2015-04-01,High


In [134]:
# Drop rows with missing values to ensure vector integrity
movies.dropna(inplace=True)
movies.reset_index(drop=True, inplace=True)

In [135]:
movie.sample(5)

,id,popularity,original_title,cast,director,genres,release_date,popularity_level
1012,10067,0.565930,The Shaggy Dog,Tim Allen|Kristin Davis|Danny Glover|Spencer B...,Brian Robbins,Comedy|Family,2006-03-09,Low
1073,13702,0.225831,A Dirty Shame,Tracey Ullman|Johnny Knoxville|Selma Blair|Chr...,John Waters,Comedy,2004-09-12,Low
155,157834,0.722877,The Zero Theorem,Christoph Waltz|David Thewlis|MÃ©lanie Thierry...,Terry Gilliam,Drama|Fantasy|Science Fiction,2014-01-02,Medium
367,45094,0.773316,Conviction,Hilary Swank|Minnie Driver|Bailee Madison|Lore...,Tony Goldwyn,Drama,2010-09-11,Medium
433,1850,0.581756,Man on the Moon,Jim Carrey|Courtney Love|Bob Zmuda|Danny DeVit...,MiloÅ¡ Forman,Comedy|Drama|Romance,1999-12-22,Low


In [136]:
def convert_pipe_to_list(text):
    """
    Parses 'Action|Adventure' strings into list ['Action', 'Adventure'].
    Robustly handles non-string inputs.
    """
    if isinstance(text, str):
        return text.split('|')
    return []

In [137]:
def split_overview(text):
    """Tokenizes the overview (plot summary) into a list of words."""
    if isinstance(text, str):
        return text.split()
    return []

In [138]:
def collapse_spaces(obj):
    """
    CRITICAL FIX: Identity Preservation.
    Converts 'Chris Pratt' -> 'ChrisPratt'.
    Prevents 'Chris Pratt' matching with 'Chris Evans' solely on 'Chris'.
    """
    if isinstance(obj, list):
        return [str(i).replace(" ", "") for i in obj]
    elif isinstance(obj, str):
        return [obj.replace(" ", "")]
    return []

In [139]:
# Convert string formats to lists
movies['genres'] = movies['genres'].apply(convert_pipe_to_list)
movies['keywords'] = movies.get('keywords', pd.Series([[]]*len(movies))).apply(convert_pipe_to_list)
movies['cast'] = movies['cast'].apply(convert_pipe_to_list)
movies['overview'] = movies['overview'].apply(split_overview)
movies['director'] = movies['director'].apply(lambda x: [x] if isinstance(x, str) else [])

In [140]:
# Limit Cast to top 3 (reduces noise from uncredited extras)
movies['cast'] = movies['cast'].apply(lambda x: x[:3])

In [141]:
# Apply Space Collapsing
for col in ['genres', 'keywords', 'cast', 'director']:
    movies[col] = movies[col].apply(collapse_spaces)

In [142]:
# Create the 'tags' column by concatenating all metadata
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['director']

In [143]:
# Create a clean DataFrame for the model
new_df = movies[['id', 'original_title', 'tags']].copy()

In [144]:
# Convert list of tags back to a single string
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

In [145]:
# Apply Stemming
ps = PorterStemmer()
def stem_text(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [146]:
new_df['tags'] = new_df['tags'].apply(stem_text)
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

In [147]:
(new_df['tags'].str.len() == 0).sum()

0

In [148]:
new_df['tags'].head(10)


0    twenty-two year after the event of jurass park...
1    an apocalypt stori set in the furthest reach o...
2    beatric prior must confront her inner demon an...
3    thirti year after defeat the galact empire, ha...
4    deckard shaw seek reveng against domin toretto...
5    in the 1820s, a frontiersman, hugh glass, set ...
6    the year is 2029. john connor, leader of the r...
7    dure a man mission to mars, astronaut mark wat...
8    minion stuart, kevin and bob are recruit by sc...
9    grow up can be a bumpi road, and it' no except...
Name: tags, dtype: object

In [149]:
# max_features=5000 is a statistical sweet spot for this data size.
# 1287 (row count) was arbitrary and too low.
cv = CountVectorizer(max_features=5000, stop_words='english')

In [150]:
# Create the vector matrix
vectors = cv.fit_transform(new_df['tags']).toarray()

In [151]:
# Calculate Cosine Similarity
similarity = cosine_similarity(vectors)

In [155]:
# 1. Define the series and query again (Global Scope)
query = 'The Dark Knight'.lower().strip() # Example movie
titles_series = new_df['original_title'].str.lower()

In [156]:
# 2. Check if the movie exists to avoid crashing
if query in titles_series.values:
    # 3. Fetch the index safely
    movie_index = titles_series[titles_series == query].index[0]
    print(f"Movie found at index: {movie_index}")
    
    # 4. Calculate distances using this index
    distances = similarity[movie_index]
    
    # 5. Get top 5 recommendations
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    print(f"Recommendations for '{new_df.iloc[movie_index].original_title}':")
    for i in movies_list:
        print(new_df.iloc[i[0]].original_title)
else:
    print(f"Movie '{query}' not found.")

Movie found at index: 453
Recommendations for 'The Dark Knight':
The Dark Knight Rises
Batman Begins
Harsh Times
Sherlock Holmes: A Game of Shadows
The Usual Suspects


In [157]:
# Output
print(f"✅ Recommendations for '{new_df.iloc[movie_index].original_title}':")
print("=" * 40)
for i in movies_list:
    print(f"• {new_df.iloc[i[0]].original_title}")

✅ Recommendations for 'The Dark Knight':
• The Dark Knight Rises
• Batman Begins
• Harsh Times
• Sherlock Holmes: A Game of Shadows
• The Usual Suspects
